In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from glob import glob

In [2]:
# sample_metadata = pd.read_table('data/sample_metadata.tsv')
# sample_metadata

In [3]:
# projects = pd.read_csv('data/projects.csv')
# projects_filtered = projects[
#     (projects['amplicon'] == 'v3-v4') & 
#     (projects['sample_type'] == 'stool') & 
#     (projects['condition'] == 'healthy') &
#     (projects['subjects'] == 'adults')
# ].reset_index(drop=True)
# projects_filtered

In [4]:
# asvs_paths = [path for path in glob('data/asv/*') if path.split('/')[-1].split('.')[0] in projects_filtered['project'].values]
# asvs_tables = [pd.read_table(path, index_col=0) for path in asvs_paths]
# asvs_table = pd.concat(asvs_tables, axis=1)
# asvs_table = asvs_table.fillna(0)
# asvs_table_rel = asvs_table.div(asvs_table.sum(axis=0), axis=1)
# asvs_table = asvs_table[
#     ~(((asvs_table_rel > 0).sum(axis=1) / asvs_table_rel.shape[1])*100 < 0.1) & 
#     ~(asvs_table_rel.sum(axis=1)*100 < 0.1)
# ]
# asvs_table.sort_index(inplace=True)
# asvs_table.to_csv('asvs_table.tsv', sep='\t')
# asvs_table

In [5]:
# asvs_paths = [path for path in glob('data/asv/*') if path.split('/')[-1].split('.')[0] in projects_filtered['project'].values]
# asvs_tables = [pd.read_table(path, index_col=0) for path in asvs_paths]
# asvs_table = pd.concat(asvs_tables, axis=0)
# asvs_table = asvs_table.fillna(0)
# asvs_table_rel = asvs_table.div(asvs_table.sum(axis=0), axis=1)
# asvs_table = asvs_table[
#     ~(((asvs_table_rel > 0).sum(axis=1) / asvs_table_rel.shape[1])*100 < 0.1) & 
#     ~(asvs_table_rel.sum(axis=1)*100 < 0.1)
# ]
# asvs_table.to_csv('asvs_table.tsv', sep='\t')
# asvs_table

In [6]:
# kos_table = pd.read_table('pred_metagenome_unstrat.tsv', index_col=0).T
# kos_table.index.name = 'sample_id'
# kos_table.columns = kos_table.columns.str.replace('ko:', '')
# kos_table = kos_table.loc[:, (kos_table.std() / kos_table.mean()) > 0.1]
# kos_table.to_csv('ko_table.tsv', sep='\t')
# kos_table

In [7]:
# import pandas as pd

# # Load ASV table (ASVs as index/rows, samples as columns)
# asv_table = pd.read_csv("asvs_table.tsv", sep="\t", index_col=0)

# # Load ASV → OTU map
# asv_to_otu = pd.read_csv("asv_to_otu_map.tsv", sep="\t")
# asv_to_otu_dict = dict(zip(asv_to_otu['ASV_ID'], asv_to_otu['OTU_ID']))

# # Map ASV row indices to OTU IDs
# asv_table['OTU_ID'] = asv_table.index.map(lambda x: asv_to_otu_dict.get(x, x))

# # Group by OTU ID and sum the counts
# otu_table = asv_table.groupby('OTU_ID').sum()

# # Save to file
# otu_table.to_csv("otus_table.tsv", sep="\t")

# Processing

In [4]:
import pandas as pd
from skbio.diversity import alpha_diversity, beta_diversity

species_df = pd.read_table("otus_table.tsv", index_col=0).T
species_df.index.name = 'sample_id'
species_df = species_df.div(species_df.sum(axis=1), axis=0)

# calculate alpha diversity
h0 = alpha_diversity("hill", species_df, ids=species_df.index, order=0)
h1 = alpha_diversity("hill", species_df, ids=species_df.index, order=1)
h2 = alpha_diversity("hill", species_df, ids=species_df.index, order=2)
 
# create a DataFrame to store the results
alpha_diversity_results = pd.DataFrame({
    "h0": h0,
    "h1": h1,
    "h2": h2
})
alpha_diversity_results.index.name = 'sample_id'
alpha_diversity_results

# save alpha diversity results to file
alpha_diversity_results.to_csv("alpha_diversity.tsv", sep="\t")


# calculate beta diversity
# beta_diversity_results = beta_diversity("braycurtis", species_df, ids=species_df.index)
# beta_diversity_results

In [6]:
import pandas as pd
from skbio.diversity import alpha_diversity, beta_diversity

gene_df = pd.read_table("ko_table.tsv", index_col=0)
gene_df.columns.name = 'KO'
gene_df = gene_df.div(gene_df.sum(axis=1), axis=0)

# calculate alpha diversity
h0 = alpha_diversity("hill", gene_df, ids=gene_df.index, order=0)
h1 = alpha_diversity("hill", gene_df, ids=gene_df.index, order=1)
h2 = alpha_diversity("hill", gene_df, ids=gene_df.index, order=2)
 
# create a DataFrame to store the results
alpha_diversity_results = pd.DataFrame({
    "h0": h0,
    "h1": h1,
    "h2": h2
})
alpha_diversity_results.index.name = 'sample_id'
alpha_diversity_results

# save alpha diversity results to file
alpha_diversity_results.to_csv("alpha_diversity_genes.tsv", sep="\t")


# calculate beta diversity
# beta_diversity_results = beta_diversity("braycurtis", species_df, ids=species_df.index)
# beta_diversity_results

In [3]:
import pandas as pd
gene_df = pd.read_table("ko_table.tsv", index_col=0)
gene_df.columns.name = 'KO'

In [4]:
gene_df

KO,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K22503,K22504,K22505,K22506,K22507,K22508,K22509,K22510,K22511,K22512
sample_id,,,,,,,,,,,,,,,,,,,,,
SRS8422867,629.21,1296.8,16671.76,0.0,4478.25,0.0,13391.20,2935.09,340.20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRS8422868,801.57,608.0,10635.90,0.0,2473.33,0.0,4874.66,3870.74,178.33,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRS8422869,6708.23,5338.5,61333.46,149.5,18390.30,0.0,56318.00,4845.73,1888.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRS8422870,6707.67,2902.5,45856.20,38.5,16252.33,0.0,29511.66,7963.00,497.33,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRS8422871,5835.84,760.0,48077.61,148.0,5257.17,0.0,12878.84,269.00,734.67,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERS5127010,57043.46,9666.5,201372.27,87.5,39816.51,0.0,35019.22,64975.76,4276.03,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERS5127011,43800.57,6027.0,149210.26,80.0,30936.43,0.0,24441.26,45399.24,3142.67,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERS5127012,35092.66,5920.0,129959.33,70.0,24913.33,0.0,22249.09,44425.34,2761.33,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
